# torch benchmark

ref) https://pytorch.org/tutorials/recipes/recipes/benchmark.html

In [ ]:
def diagmul(mat, diag):
    return mat * diag

In [ ]:
import torch
import torch.utils.benchmark as benchmark

A = torch.randn(256, 256)
d = torch.randn(256)
num_threads = torch.get_num_threads()
t0 = benchmark.Timer(
    stmt="torch.matmul(A, torch.diag(d))", globals={"d": d, "A": A}, num_threads=num_threads
).blocked_autorange(min_run_time=1)
t1 = benchmark.Timer(
    stmt="diagmul(d, A)",
    setup="from __main__ import diagmul",
    globals={"d": d, "A": A},
    num_threads=num_threads,
).blocked_autorange(min_run_time=1)

In [ ]:
(A * d).shape

In [ ]:
print("Matmul: ", t0)
print("Diagmul: ", t1)

In [ ]:
t2 = benchmark.Timer(
    stmt="d.mul(A)", globals={"d": d, "A": A}, num_threads=num_threads
).blocked_autorange(min_run_time=1)
print("Matmul: ", t2)

In [ ]:
from itertools import product

results = []
sizes = [1, 64, 1024]
for m, n in product(sizes, sizes):
    # label and sub_label are the rows
    # description is the column
    label = "Diagonal matrix multiplication"
    sub_label = f"[{m}, {n}]"
    A = torch.rand((m, n))
    d = torch.rand(n)
    for num_threads in [1, 4, 8]:
        results.append(
            benchmark.Timer(
                stmt="torch.matmul(A, torch.diag(d))",
                globals={"d": d, "A": A},
                num_threads=num_threads,
                label=label,
                sub_label=sub_label,
                description="mm",
            ).blocked_autorange(min_run_time=1)
        )
        results.append(
            benchmark.Timer(
                stmt="diagmul(A,d)",
                setup="from __main__ import diagmul",
                globals={"d": d, "A": A},
                num_threads=num_threads,
                label=label,
                sub_label=sub_label,
                description="vm",
            ).blocked_autorange(min_run_time=1)
        )

compare = benchmark.Compare(results)
compare.print()

In [ ]:
compare